In [ ]:
%load_ext autoreload
%autoreload 2
#%matplotlib notebook

In [ ]:
import pdb
import math
from scipy import integrate
from scipy import special
import numpy as np
from cmath import *
import matplotlib.pyplot as plt
from IPython.display import display, Math, Latex
from patch_square import *
np.seterr(all='raise')
special.seterr(all='raise')
warpSize = 32
blockSize = 1024
fdr = '/root/autodl-tmp/public/resource'
from patch_geo_func import *

In [ ]:
den_area = 850 # mm^2 O'Kusky 1982 for the neuronal density we use later
# exp. measurement
darea = lambda E: 140*(0.78+E)**(-2.2) # Van Essen 1984 Vision Research "standard" map, 5232x1024
area_slice = lambda E: np.pi*E*darea(E)
# model fit
a = 0.635
b = 96.7
k = np.sqrt(140)*0.873145
s1 = 0.76
s2 = 0.1821
model_block = lambda p, e: model_block_ep(e,p,k,a,b,s1,s2)
half_stripe_width = 0 # to be read from elastic net simulation result

def model_slice(e):
    dslice = lambda p: model_block(p, e)
    r = integrate.quad(dslice, -np.pi/2, np.pi/2)
    return r[0]

ecc = np.hstack((np.linspace(0,2.5,50), np.linspace(2.5,90,100)))
ecc = ecc[1:]
fig = plt.figure('model_compare')
ax = fig.add_subplot(111)
model_s = np.array([model_slice(e) for e in ecc])
exp_s = np.array([area_slice(e) for e in ecc])
ax.plot(ecc, exp_s,'r')
ax.plot(ecc, model_s,'b')
ax2 = ax.twinx()
ax2.plot(ecc, np.abs(exp_s - model_s)/exp_s)

In [ ]:
# total v1 size
r = integrate.quad(area_slice,0,90)
v1_size_exp = r[0]
r = integrate.dblquad(model_block,0,90,-pi/2,pi/2)
v1_size_model = r[0]
display(Latex(rf'Size of V1 = {v1_size_exp:.3f} $mm^2$ (exp.) ~ {v1_size_model:.3f} $mm^2$ (model) ({v1_size_model*100/v1_size_exp:.3f}%)'))
ecc = 1.0 ########################################### <- set here 0.08 checked
r = integrate.quad(area_slice,0,ecc)
area_exp = r[0]
r = integrate.dblquad(model_block,0,ecc,-np.pi/2,np.pi/2)
area_model = r[0]
display(Latex(rf'A patch of v1 from eccentricity {0} to {ecc} degree yields {area_exp:.3f}  $mm^2$ (exp.) ~ {area_model:.3f}  $mm^2$ (model) ({area_model*100/area_exp:.3f}%)'))
model_ratio = den_area/v1_size_model
print(f'ratio: {model_ratio:.3f}')

In [ ]:
print('O''Kusky 1982; Kelly & Hawken 2017; M.Schmidt 2018, assuming constant density near the fovea')
print('##### L4 #####')
surface_den_L4Cbeta = 30000*model_ratio # per mm^2
display(Latex(rf'Rescale density 30000 per $mm^2$ in O\'Kusky 1982 to {surface_den_L4Cbeta:.3f} per $mm^2$ to keep constant number of neurons'))
assert(30000*den_area == surface_den_L4Cbeta*v1_size_model)
_n_L4Cbeta = area_model*surface_den_L4Cbeta # 32^4 * 10 ~ 10.4 million
n_L4Cbeta = int((_n_L4Cbeta+blockSize-1)//blockSize*blockSize)
nblock_L4Cbeta = n_L4Cbeta//blockSize
block_area_L4Cbeta = area_model/nblock_L4Cbeta
L4_area = area_model
print(f'Rounding number of neurons in L4Cbeta from {_n_L4Cbeta:.3f} to {n_L4Cbeta} = {nblock_L4Cbeta} x {blockSize}')
display(Latex(rf'area per block = {block_area_L4Cbeta:.3f} $mm^2$'))
display(Latex(rf'model density = {n_L4Cbeta/L4_area:.3f} per $mm^2$\n'))
print('##### L2/3 #####')
surface_den_L23 = 18000*model_ratio # per mm^2
display(Latex(rf'Rescale density 18000 per $mm^2$ in O\'Kusky 1982 to {surface_den_L23:.3f} per $mm^2$ to keep constant number of neurons'))
_n_L23 = area_model*surface_den_L23
n_L23 = int((_n_L23+blockSize-1)//blockSize*blockSize)
nblock_L23 = n_L23//blockSize
block_area_L23 = area_model/nblock_L23
L23_area = area_model
print(f'Rounding number of neurons in L2/3 from {_n_L23:.3f} to {n_L23} = {nblock_L23} x {blockSize}')
display(Latex(f'area per block = {block_area_L23:.3f} $mm^2$'))
display(Latex(f'model density = {n_L23/L23_area:.3f} per $mm^2$'))

In [ ]:
nblock = nblock_L4Cbeta - 25
theme = f'{area_model:.0f}mm2-{nblock}x{blockSize}'
print(theme)

In [ ]:
fig, pos, xlim, ylim = plot_patch(a,b,k,ecc,half_stripe_width,nblock,ax=None,skip=602,s1=0.76,s2=0.1821,ret_fig=True,blockSize=blockSize)
#fig, pos = plot_patch(a,b,k,ecc,L4_area/32,32,ax=None,skip=602,s1=0.76,s2=0.1821,ret_fig=True,blockSize=1024)

In [ ]:
fig.savefig(f'{fdr}/patch_{theme}.png', dpi = 600)
dim = 2
pos_filename = f'{fdr}/2Dpos_{theme}.bin'
position_discrimination_digits = 1
with open(pos_filename, 'wb') as f:
    np.array([nblock_L4Cbeta, blockSize, dim]).astype('u4').tofile(f)        
    pos[:,:2,:].tofile(f) #f8
    
dim = 3
pos_filename = f'{fdr}/3Dpos_{theme}.bin'
position_discrimination_digits = 1
with open(pos_filename, 'wb') as f:
    np.array([nblock_L4Cbeta, blockSize, dim]).astype('u4').tofile(f)        
    pos.tofile(f) #f8

In [ ]:
nblock = 56
blockSize = 1024
low_theme = f'{area_model:.0f}mm2-{nblock}x{blockSize}'

In [ ]:
fig, pos, xlim, ylim = plot_patch(a,b,k,ecc,half_stripe_width,nblock,ax=None,skip=32,s1=0.76,s2=0.1821,ret_fig=True,blockSize=blockSize)

In [ ]:
fig.savefig(f'{fdr}/low_{low_theme}.png', dpi = 600)
dim = 2
pos_filename = f'{fdr}/low_2Dpos_{low_theme}.bin'
with open(pos_filename, 'wb') as f:
    np.array([nblock, blockSize, dim]).astype('u4').tofile(f)        
    pos[:,:2,:].tofile(f) #f8
    
dim = 3
pos_filename = f'{fdr}/low_3Dpos_{low_theme}.bin'
with open(pos_filename, 'wb') as f:
    np.array([nblock, blockSize, dim]).astype('u4').tofile(f)
    pos.tofile(f)

characteristic length =

In [ ]:
cl = np.sqrt(block_area_L4Cbeta) * 1000
display(Latex(f'{cl:.3f} $\mu m$'))

In [ ]:
md = np.power(cl*cl*100/1024,1/3)
display(Latex(f'~204x204x100 $\mu m^3$ (1024 neurons) per block, average inter-neuron(soma)-distance = {md:.3f} $\mu m$'))

conduction velocity ~ 1m/s, being unmylineated horizontal connections  
(0.3m/s for L2/3 and upper L4) <i>Girard et al J Neurophysiol 2001</i>  
minmum delay = 0.016 ms, no spike-correction is needed for time step (dt) < minimum delay
100Hz instantaneous firing rate, 1000 connections leads to ~ 1 spike every 0.01ms. 

In [ ]:
dt = 1 #ms
nob = 1000/cl
print(nob)

dt = 1ms means neurons in the nearest 5~6 blocks needs to be considered for spike correction  
dt = 0.125ms (1/8) suffice for spike correction within block, #ASSUMPTION# nearfield spike correction only (neurons at block boundarys are neglected) spikes send to the other blocks are resolved after the current step.

#dt = 0.125ms is thus limited by the number of threads per block in NVIDIA GPU  
To further increase dt, will need a cross block spike-correction with CPU, or wait for an update from NVIDIA, otherwise results in an extra increase in spike-correction error, maybe tolerable.

In [ ]:
time = 500
dt = 0.0625
batch_time_per_dt = 500e-6 #sec
nstep = time//dt
blocks = 10240
resident_blocks = 8
time_cost = int(blocks/resident_blocks) * batch_time_per_dt * nstep
print('simulate for', time ,'ms with dt =', dt, 'ms cost', time_cost/3600,'h')

In [ ]:
from scipy import integrate
# Maplpeli et al 1996
parvo_den_alt = lambda E: 1011688*(E+2.9144)**(-2.6798) # cells/deg^2
# offset (transition between crossed and uncrossed LGN Chalupa and Lia 1991, Lavidor and Walsh 2004)
#offset = 0.5/
#parvo_slice = lambda E: (np.pi+offset)*E*parvo_den_alt(E)
parvo_slice = lambda E: np.pi*E*parvo_den_alt(E)
ecc_Connelly = 2.5
r = integrate.quad(parvo_slice, 0, ecc_Connelly)
nparvo_connelly = round(r[0])
print(f'{nparvo_connelly} parvo cells inside {ecc_Connelly} in total > 110,000 connelly and van essen 1984')
magno_den_alt = lambda E: 2620.2*((E-1.8322)**2+5.5638)**(-0.8012) # cells/deg^2

In [ ]:
# in the cortex sheet
# R dendritic spread of L4 neuron
# r dendritic spread of LGN neuron
R_beta = 0.05 # mm Lund 1973
R_alpha = 0.1 # mm
r_parvo = 0.1 # mm Blasdel and lund 1983
r_magno = 0.25 # mm
def M_V1(e):
    dslice = lambda p: model_block(p, e)
    meanM = integrate.quad(dslice, -np.pi/2, np.pi/2)
    return meanM[0]/np.pi

# lgn per cortex cell = den_alt(ecc) * (R+r)^2 * darea(ecc)
#parvo_beta = lambda e: parvo_den_alt(e) * np.power((R_beta + r_parvo),2) * M_V1(e)
spread = np.power((R_beta + r_parvo),2)
parvo_beta = lambda e: parvo_den_alt(e) * spread / M_V1(e)
#parvo_beta = lambda e: 24610 * M_V1(e) / parvo_den_alt(e)

x_ecc = x_ep(ecc,0,k,a,b) + r_parvo
ecc0 = e_x(x_ecc,k,a,b)
r = integrate.quad(parvo_slice, 0, ecc0)
nparvo0 = round(r[0])
print(f'V1 occupies {ecc} eccentricity, parvo LGN occupies {ecc0} eccentricity')
print(f'{nparvo0} parvo cells within {ecc0} degree')
#ratio_L56 = 0.7 #L5,6 only is not correct, use 1.0, sked Bob
ratio_CI = 0.66 # C:I ~ 4:1 (Connolly and Van Essen 1984) within 2.5 degree
ecc_arr = np.linspace(0,ecc0,101)
ecc_arr = ecc_arr[1:]
fig = plt.figure('parvo per 4Cbeta',  dpi = 60)
ax = fig.add_subplot(111)
ax.set_title(f'parvo per 4Cbeta, spread = {spread:.4f}')
ax.plot(ecc_arr, [parvo_beta(e)*ratio_CI for e in ecc_arr], ':b')
ax.plot(ecc_arr, [parvo_beta(e)*(1-ratio_CI) for e in ecc_arr], ':r')
ax.set_xscale('log')

In [ ]:
def half_poisson_disk(n, radius, seed, jiggle):
    n0 = int(np.round(4/np.pi*2*n))
    ratio = 1
    pos0 = fast_poisson_disk2d(1, n0, ratio = 1.0, m = 32, jiggle = jiggle, seed = seed, plot = False)
    while pos0.shape[0] < n0:
        seed += 1
        ratio *= n0/pos0.shape[0]
        pos0 = fast_poisson_disk2d(1, int(n0 * ratio), ratio = 1, m = 32, jiggle = jiggle, seed = seed, plot = False)
    
    pos0 -= 0.5
    pick = pos0[:,0] > 0
    pos0 = pos0[pick,:]
    r = np.linalg.norm(pos0, axis = 1)
    pick = np.argpartition(r, n)[:n]
    pos = pos0[pick,:].copy()
    r = r[pick]
    max_r = np.max(r)
    ratio = radius / max_r
    return pos*ratio, r*ratio

def half_poisson_disk_with_gradient(density, n, radius, jiggle = 0.05, seed = 3475286, plot = False):
    uniform_pos, r = half_poisson_disk(n, radius, seed, jiggle)
    linear_density = lambda x: np.pi*x*density(x)
    float_n = integrate.quad(linear_density, 0, radius)[0]
    corrected_density = lambda x: linear_density(x) * n/float_n
    decc = lambda x: np.sqrt(1/(density(x)*n/float_n))
    ecc_range = np.linspace(0, ecc0, 32)
    if plot:
        fig = plt.figure(figsize = (10, 4), dpi = max(n//32, 200))
        ax = fig.add_subplot(131)
        ax.plot(ecc_range, corrected_density(ecc_range), '-k')
        ax = ax.twinx()
        ax.plot(ecc_range, decc(ecc_range), '-r')

    pos = uniform_pos.copy()
    acc_r = radius
    polar_range = np.linspace(-np.pi/2,np.pi/2,32)
    while acc_r > 0:
        if acc_r - decc(acc_r) < 0:
            acc_r = acc_r/2
        else:
            acc_r -= decc(acc_r)
        #ax.plot(acc_r*np.cos(polar_range), acc_r*np.sin(polar_range), '-k', lw = 0.5)
        m = int(round(integrate.quad(corrected_density, 0, acc_r)[0]))
        if m == 0:
            break
        pick = np.argpartition(r, m)[:m]
        #print(acc_r, np.max(r[pick]), m)
        if np.max(r[pick]) > acc_r:
            pos[pick,:] *= acc_r/np.max(r[pick])
            r = np.linalg.norm(pos, axis = 1)
        #print(r.min(), r.max())
    if plot:    
        ax = fig.add_subplot(132)
        ax.plot([pos[:,0], uniform_pos[:,0]], [pos[:,1], uniform_pos[:,1]], '-k', lw = 0.1)
        ax.plot(uniform_pos[:,0], uniform_pos[:,1], ',b')
        ax.plot(pos[:,0], pos[:,1], ',r')
        ax.set_aspect('equal')
        ax = fig.add_subplot(133)
        ax.plot(pos[:,0], pos[:,1], ',r')
        ax.set_aspect('equal')
        fig.tight_layout(w_pad = 2)
    return pos.T, uniform_pos.T

# test poisson disk
#n = 1024
#radius = ecc0
#n0 = int(np.round(4/np.pi*2*n))
#seed = 3475285
#fast_poisson_disk2d(1, n0, ratio = 1.0, m = 32, attempts = 1, attempts0 = 1, jiggle = 0.05, seed = seed, plot = True);
#pos, uniform_pos = half_poisson_disk_with_gradient(parvo_den_alt, n, radius, jiggle = 0, plot = True)

In [ ]:
seed = 347521
# Excluding S_on, S_off from LGN Koniocellular layers to L1, L2/3 for now
# C-On:Off ~ 3:1; I-On:Off ~ 2:1 (Wiesel and Hubel 1966, Schiller-and-Malpeli-1978, Derrington and Lennie 1984, Martin-and-Lee 2014), cone-to-ganglion ~ 1:1 (McMahon et al., 2000)
# Red:Green, 1.0 central (Bowmaker 2003, cite-only.bib) 1.6 overall(Deeb et al., 2000, cite-only.bib)
ratio_Lon = 0.25
ratio_Loff = 0.25
ratio_Mon = 0.25
ratio_Moff = 0.25
assert(ratio_Lon + ratio_Loff + ratio_Mon + ratio_Moff == 1)
nparvo_C6 = int(np.round(nparvo0 * ratio_CI))
nparvo_C6_Lon = int(np.round(nparvo_C6 * ratio_Lon))
nparvo_C6_Loff = int(np.round(nparvo_C6 * ratio_Loff))
nparvo_C6_Mon = int(np.round(nparvo_C6 * ratio_Mon))
nparvo_C6_Moff = nparvo_C6 - nparvo_C6_Lon - nparvo_C6_Loff - nparvo_C6_Mon

nparvo_I5 = int(np.round(nparvo0 * (1-ratio_CI)))
nparvo_I5_Lon = int(np.round(nparvo_I5 * ratio_Lon))
nparvo_I5_Loff = int(np.round(nparvo_I5 * ratio_Loff))
nparvo_I5_Mon = int(np.round(nparvo_I5 * ratio_Mon))
nparvo_I5_Moff = nparvo_I5 - nparvo_I5_Lon - nparvo_I5_Loff - nparvo_I5_Mon

nparvo = nparvo_I5 + nparvo_C6
print(nparvo_I5, nparvo_C6)
pos = np.empty((2,nparvo))
uniform_pos = np.empty((2,nparvo))
RGC_type = np.empty(nparvo, dtype = 'u4')

i = 0
print(f'I5_Lon: {nparvo_I5_Lon}')
RGC_type[i:i+nparvo_I5_Lon] = 0
pos[:, i:i+nparvo_I5_Lon], uniform_pos[:, i:i+nparvo_I5_Lon] = half_poisson_disk_with_gradient(parvo_den_alt, nparvo_I5_Lon, ecc0, seed)
i += nparvo_I5_Lon
print(f'I5_Loff: {nparvo_I5_Loff}')
RGC_type[i:i+nparvo_I5_Loff] = 1
pos[:, i:i+nparvo_I5_Loff], uniform_pos[:, i:i+nparvo_I5_Loff] = half_poisson_disk_with_gradient(parvo_den_alt, nparvo_I5_Loff, ecc0, seed*2)
i += nparvo_I5_Loff
print(f'I5_Mon: {nparvo_I5_Mon}')
RGC_type[i:i+nparvo_I5_Mon] = 2
pos[:, i:i+nparvo_I5_Mon], uniform_pos[:, i:i+nparvo_I5_Mon] = half_poisson_disk_with_gradient(parvo_den_alt, nparvo_I5_Mon, ecc0, seed*3)
i += nparvo_I5_Mon
print(f'I5_Moff: {nparvo_I5_Moff}')
RGC_type[i:i+nparvo_I5_Moff] = 3
pos[:, i:i+nparvo_I5_Moff], uniform_pos[:, i:i+nparvo_I5_Moff] = half_poisson_disk_with_gradient(parvo_den_alt, nparvo_I5_Moff, ecc0, seed*4)
i += nparvo_I5_Moff

print(f'C6_Lon: {nparvo_C6_Lon}')
RGC_type[i:i+nparvo_C6_Lon] = 0
pos[:, i:i+nparvo_C6_Lon], uniform_pos[:, i:i+nparvo_C6_Lon] = half_poisson_disk_with_gradient(parvo_den_alt, nparvo_C6_Lon, ecc0, seed*5)
i += nparvo_C6_Lon
print(f'C6_Loff: {nparvo_C6_Loff}')
RGC_type[i:i+nparvo_C6_Loff] = 1
pos[:, i:i+nparvo_C6_Loff], uniform_pos[:, i:i+nparvo_C6_Loff] = half_poisson_disk_with_gradient(parvo_den_alt, nparvo_C6_Loff, ecc0, seed*6) 
i += nparvo_C6_Loff
print(f'C6_Mon: {nparvo_C6_Mon}')
RGC_type[i:i+nparvo_C6_Mon] = 2
pos[:, i:i+nparvo_C6_Mon], uniform_pos[:, i:i+nparvo_C6_Mon] = half_poisson_disk_with_gradient(parvo_den_alt, nparvo_C6_Mon, ecc0, seed*7)
i += nparvo_C6_Mon
print(f'C6_Moff: {nparvo_C6_Moff}')
RGC_type[i:i+nparvo_C6_Moff] = 3
pos[:, i:i+nparvo_C6_Moff], uniform_pos[:, i:i+nparvo_C6_Moff] = half_poisson_disk_with_gradient(parvo_den_alt, nparvo_C6_Moff, ecc0, seed*8)
i += nparvo_C6_Moff
assert(i == nparvo)

In [ ]:
fig = plt.figure(dpi = nparvo//50)
marker = 'o'
ms = 0.01
i = 0
ax = fig.add_subplot(121)
ax.plot(pos[0, i:i+nparvo_I5_Lon], pos[1, i:i+nparvo_I5_Lon], marker, ms = ms, color = 'red')
i += nparvo_I5_Lon
ax.plot(pos[0, i:i+nparvo_I5_Loff], pos[1, i:i+nparvo_I5_Loff], marker, ms = ms, color = 'darkred')
i += nparvo_I5_Loff
ax.plot(pos[0, i:i+nparvo_I5_Mon], pos[1, i:i+nparvo_I5_Mon], marker, ms = ms, color = 'limegreen')
i += nparvo_I5_Mon
ax.plot(pos[0, i:i+nparvo_I5_Moff], pos[1, i:i+nparvo_I5_Moff], marker, ms = ms, color = 'darkgreen')
i += nparvo_I5_Moff
ax.set_aspect('equal')
ax = fig.add_subplot(122)
ax.plot(pos[0, i:i+nparvo_C6_Lon], pos[1, i:i+nparvo_C6_Lon], marker, ms = ms, color = 'red')
i += nparvo_C6_Lon
ax.plot(pos[0, i:i+nparvo_C6_Loff], pos[1, i:i+nparvo_C6_Loff], marker, ms = ms, color = 'darkred')
i += nparvo_C6_Loff
ax.plot(pos[0, i:i+nparvo_C6_Mon], pos[1, i:i+nparvo_C6_Mon], marker, ms = ms, color = 'limegreen')
i += nparvo_C6_Mon
ax.plot(pos[0, i:i+nparvo_C6_Moff], pos[1, i:i+nparvo_C6_Moff], marker, ms = ms, color = 'darkgreen')
i += nparvo_C6_Moff
ax.set_aspect('equal')
assert(i == nparvo)

In [ ]:
prec = 'float32'
#for assign_attr.py
LGN_theme = 'type_mosaic'
parvo_pos_I5_file = f'{fdr}/parvo_pos_I5-{LGN_theme}.bin'
with open(parvo_pos_I5_file, 'wb') as f:
    np.array([nparvo_I5], dtype = 'u4').tofile(f)
    np.array([ecc0], dtype = 'f8').tofile(f)
    uniform_pos[:,:nparvo_I5].tofile(f)
    RGC_type[:nparvo_I5].tofile(f)

parvo_pos_C6_file = f'{fdr}/parvo_pos_C6-{LGN_theme}.bin'
with open(parvo_pos_C6_file, 'wb') as f:
    np.array([nparvo_C6], dtype = 'u4').tofile(f)
    np.array([ecc0], dtype = 'f8').tofile(f)
    uniform_pos[:,nparvo_I5:].tofile(f)
    RGC_type[nparvo_I5:].tofile(f)

In [ ]:
def cart2polar(cart):
    r = np.linalg.norm(cart, axis = 0)
    polar = np.arctan2(cart[1,:], cart[0,:])
    pol = np.vstack((polar, r))
    print(pol.shape)
    return pol 
"""
parvo_pos_I5_file = 'parvo_pos_I5.bin'
with open(parvo_pos_I5_file, 'wb') as f:
    np.array([nparvo_I5], dtype = 'u4').tofile(f)
    RG_OnOff_I5.astype('int32').tofile(f)
    parvo_pos_I5.astype(prec).tofile(f)
    
parvo_pos_C6_file = 'parvo_pos_C6.bin'
with open(parvo_pos_C6_file, 'wb') as f:
    np.array([nparvo_C6], dtype = 'u4').tofile(f)
    RG_OnOff_C6.astype('int32').tofile(f)
    parvo_pos_C6.astype(prec).tofile(f)
""" 
prec = 'float32'
LR_merged_file = f'{fdr}/parvo_float-{LGN_theme}.bin'
with open(LR_merged_file, 'wb') as f:
    # Ipsi first
    np.array([nparvo_I5], dtype = 'u4').tofile(f)
    np.array([nparvo_C6], dtype = 'u4').tofile(f)
    np.array([ecc0], dtype = prec).tofile(f)
    # cart
    np.array([0, ecc0, -ecc0, 2*ecc0], dtype = prec).tofile(f)
    pos.astype(prec).tofile(f)
    # type
    RGC_type.tofile(f)
    # polar
    polar_pos = cart2polar(pos)
    polar_pos.astype(prec).tofile(f)
    
print(f'#ipsi = {nparvo_I5}, #contra = {nparvo_C6}')

In [ ]:
def area(raxn, rden, d):
    area = np.zeros(d.size)
    minr = max(raxn,rden) - min(raxn,rden)
    maxr = max(raxn,rden) + min(raxn,rden)
    area[d <= minr] = np.power(min(raxn,rden),2)*np.pi
    mid_pick = np.logical_and(minr < d, d < maxr)
    d_mid = d[mid_pick]
    cos_theta_axn = (raxn*raxn + d_mid*d_mid - rden*rden)/(2*raxn*d_mid)
    cos_theta_den = (rden*rden + d_mid*d_mid - raxn*raxn)/(2*rden*d_mid)
    seg_axn = np.arccos(cos_theta_axn)*raxn*raxn
    seg_den = np.arccos(cos_theta_den)*rden*rden

    chord_axn = np.sqrt(raxn*raxn - np.power(cos_theta_axn*raxn,2)) * raxn * cos_theta_axn
    chord_den = np.sqrt(rden*rden - np.power(cos_theta_den*rden,2)) * rden * cos_theta_den
    area[mid_pick] = seg_axn+seg_den-chord_axn-chord_den
    assert(np.sum(area<0) == 0)
    return area

def base(raxn, rden, d):
    #subr = d-raxn
    #subr[subr<0] = 0
    #supr = d+raxn
    #supr[supr>rden] = rden
    #base = np.pi*(np.power(supr,2) - np.power(subr,2))
    base = np.pi*(np.power(raxn,2))
    return base

In [ ]:
raxn = 100 #um
rden = 100
nd = 100
dd = (raxn+rden)/nd
dee = np.linspace(0,raxn+rden-dd,nd-1)

fig = plt.figure('prob')
ax = fig.add_subplot(111)
ratio = 0.095
ax.plot(dee,  area(raxn,rden,dee)/base(raxn,rden,dee)*ratio)
x = np.array([12.5, 37.5, 75, 150])
y = np.array([8.8, 7.0, 5.0, 1.3])/100
ax.plot(x,y)
ax.set_xlim(x[0],x[-1])

In [ ]:
raxn = 100
rden = 80
nd = 100
dd = (raxn+rden)/nd
dii = np.linspace(0, raxn+rden-dd,nd-1)
fig = plt.figure('prob')
ax = fig.add_subplot(211)
y = area(raxn,rden,dii)/base(raxn,rden,dii)
baseline = -0.1
ratio = 1
ax.plot(dii,y*ratio+baseline)
x = [12.5, 37.5, 75]
y = [0.72, 0.50, 0.55]
yr = [0.51, 0.33, 0.27]
ax.plot(x,y)
ax.plot(x,yr)
ax.set_xlim(x[0],x[-1])

In [ ]:
#Connelly-Van_Essen-1984 
Ap = 8.37
Bp = 1.28
Cp = -1.96
parvo_den = lambda E: Ap*(E+Bp)**Cp*10000 # cells/deg^2
Am = 3.52
Bm = 3.1
Cm = -1.56
magno_den = lambda E: Am*(E+Bm)**Cm*1000# cells/deg^2
deg = 5
#consistent Maplpeli et al 1996
parvo_den_alt = lambda E: 1011688*(E+2.9144)**(-2.6798)
magno_den_alt = lambda E: 2620.2*((E-1.8322)**2+5.5638)**(-0.8012)
fig = plt.figure('lgn cell areal mf', dpi=150)
ax = fig.add_subplot(221)
ecc = np.arange(0,90)
ax.plot(ecc, magno_den(ecc))
ax.plot(ecc, magno_den_alt(ecc),':')
ax.set_title(f'{magno_den(deg):.1f}, {magno_den_alt(deg):.1f}')
ax.set_ylabel('magno den')
ax.set_yscale('log')
ax = fig.add_subplot(222)
ax.plot(ecc, parvo_den(ecc))
ax.plot(ecc, parvo_den_alt(ecc),':')
ax.set_yscale('log')
ax.set_title(f'{parvo_den(deg):.1f}, {parvo_den_alt(deg):.1f}')
ax.set_ylabel('parvo den') 
est_4Cbeta_over_4A = 0.6
#blasdel and lund 1983,
magno_spread = 0.5 # mm (mean) 
parvo_spread = 0.1 # mm (upper limit)
surface_den_L4Calpha = 17300
magno_spread_reach_max = surface_den_L4Calpha*magno_spread**2
parvo_spread_reach_max = surface_den_L4Cbeta*parvo_spread**2
ax = fig.add_subplot(223)
ax.plot(ecc, magno_den(ecc)/darea(ecc,0)*magno_spread**2)
ax.plot(ecc, magno_den_alt(ecc)/darea(ecc,0)*magno_spread**2, ':')
ax.set_ylabel('magno per 4Calpha')
ax.set_title(f'{magno_den(deg)/darea(deg,0)*magno_spread**2:.1f}, {magno_den_alt(deg)/darea(deg,0)*magno_spread**2:.1f}')
ax = fig.add_subplot(224)
ax.plot(ecc, parvo_den(ecc)/darea(ecc,0)*parvo_spread**2)
ax.plot(ecc, parvo_den_alt(ecc)/darea(ecc,0)*parvo_spread**2, ':')
ax.set_ylabel('parvo per 4A and/or 4Cbeta')
ax.set_title(f'{parvo_den(deg)/darea(deg,0)*parvo_spread**2:.1f}, {parvo_den_alt(deg)/darea(deg,0)*parvo_spread**2:.1f}')
plt.tight_layout()

In [ ]:
fig.savefig(f'{fdr}/upper_limit_of_nLGN_per_V1-{LGN_theme}.png')

In [ ]:
deg = 5
p_den = lambda E: 110000*(E+1.28)**(-1.96) #cell/deg^2
m_den = lambda E: 4600*(E+3.12)**(-1.56) #cell/deg^2
print(p_den(deg),m_den(deg))
areal_cmf = lambda E: 12.2**2*(E+0.94)**(-2.0) # mm^2/deg^2
print(areal_cmf(deg))
parvo_per_p = p_den(deg)/areal_cmf(deg)
print(f'{parvo_per_p:.1f} P cells/mm^2 to 4Cbeta + 4A')
magno_per_m = m_den(deg)/areal_cmf(deg)
print(f'{magno_per_m:.1f} M cells/mm^2 to 4Calpha')

In [ ]:
fig = plt.figure('Schein')
ax = fig.add_subplot(111)
ax.plot(ecc, p_den(ecc)/areal_cmf(ecc))
ax.plot(ecc, m_den(ecc)/areal_cmf(ecc),':')